## Notebook to calculate monthly authorized users - Copy

This notebook calculates monthly authorized users based on history of `/v1` and `/v2` messages retrieved through the `/logs` API. The count is for all workspaces and/or skills for one service instance of Assistant. There are two options for use:
  - Process a list of files in a directory (JSON output from calls to `/logs` API in a directory. For this set `LOGS_DIR_PATH` in the next cell to point to the directory. If this value is set, the notebook will not attempt to access the service instance by API.
  - Access a service instance directly to extract the logs using the `/logs` API and then process. For this you need the `API key` and `Assistant URL` values from the settings of one of the Assistants or Skills. We're only using the URL up to the instance ID, so asny of the URLs and API keys listed on your assistant settings will work. Set them in the next cell below, or set these values to `''` and be prompted when the notebook runs. 

You can also get flattened log output in CSV format by specifying `EXPORT_FILE_PATH` in the cell below. If you do not want this output, set the value to `''` or `None`.

Code for this lives at https://github.com/kineticsquid/Utilities.git.

In [ ]:
LOGS_DIR_PATH = '/Users/jk/Temp/Oficinas-Logs/Oficinas3'
api_key = ''
assistant_url = 'https://api.us-east.assistant.watson.cloud.ibm.com/instances/96a44bf2-df75-4683-852d-1187573b8948'
EXPORT_FILE_PATH = 'output.csv'

In [ ]:
%pwd

In [ ]:
import string
import re
import os
import requests
import datetime
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
%cd ..
import get_logs

In [ ]:
if LOGS_DIR_PATH is not None and len(LOGS_DIR_PATH) > 0:
    files = os.listdir(LOGS_DIR_PATH)
    all_logs = []
    for file_name in files:
        abs_path = os.path.join(LOGS_DIR_PATH, file_name)
        if os.path.isfile(abs_path) and file_name[0] != '.':
            file = open(abs_path, 'r')
            logs = json.load(file)
            all_logs += logs
            file.close()
            print('Added %s log entries from file \'%s\'.' % (len(logs), file_name))
else:
    if api_key is None or len(api_key) == 0:
        api_key = input("Enter Assistant API key")
    if assistant_url is None or len(assistant_url) == 0:
        assistant_url = input("Enter assistant or skill /v1 or /v2 API URL")
    api_regex = r'https:\/\/api..*.com\/instances\/([a-zA-Z0-9]|-)+'
    api_url = re.search(api_regex, assistant_url).group()

    os.environ['APIKEY'] = api_key
    os.environ['API_URL'] = api_url
    all_logs = get_logs.get_all_logs()
print('\n%s log entries to process.' % len(all_logs))

In [ ]:
user_detail, authorized_user_counts_by_month, unique_authorized_users_by_month = get_logs.calculate_authorized_users(all_logs)

In [ ]:
user_count_df = pd.DataFrame(authorized_user_counts_by_month)
user_count_df.plot.bar(stacked=True, figsize=(10, 8), title='Authorized Users by Month', fontsize=15) 
plt.show()

user_count_df.loc["Total"] = user_count_df.sum(axis=0)
user_count_df.loc[:, "Total"] = user_count_df.sum(axis=1)
user_count_df = user_count_df.style.set_properties(**{'font-size': '12pt',})
user_count_df

In [ ]:
detail_df = pd.DataFrame(user_detail)
if EXPORT_FILE_PATH is not None and len(EXPORT_FILE_PATH) > 0:
    detail_df.to_csv(EXPORT_FILE_PATH)

In [ ]:
detail_df